In [8]:
from dotenv import load_dotenv

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import autogen

load_dotenv()

config_list_file_name = ".config_list"

gemini_config_list = config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper_review",
        "use_docker": False,
    }

user_proxy = UserProxyAgent(
    name="UserProxyAgent",
    system_message="A human user who needs a technical review document.",
    code_execution_config=code_execution_config,
    human_input_mode="NEVER",
)

# Create the researcher agent
researcher = AssistantAgent(
    name="Researcher",
    system_message=f"""You are a researcher specialized in finding relevant research papers.
    You're expert at retrieving and analyzing scientific publications.
    When asked to search for papers for a given topic.
        1. Implement functions for finding top 5 most relevant publications within recent years utilizing Python semanticscholar.
        2. You may need to install Python modules i.e. semanticscholar usin bash pip as a pre-requisite step.
           - Here is an example of how to search with semanticscholar:
           ```from semanticscholar import SemanticScholar
            sch = SemanticScholar()
            search_results = sch.search_paper(query, limit=limit)

            papers = []
            count = 0
            for paper in search_results:
                if count >= limit:
                    break
                papers.append({{
                    "title": 'paper.title,'
                    "authors": '[author.name for author in paper.authors],'
                    "year":'paper.year,'
                    "abstract": 'paper.abstract,'
                    "sch_url": 'paper.url,'
                    "citationCount": 'paper.citationCount,'
                    "url": "https://doi.org/" + paper.externalIds["DOI"],
                    "venue": 'paper.venue,'
                    "volume": 'paper.journal["volume"]'
                }})
                count += 1```
            
            Keep in mind the search_paper API return PaginatedResults so it might got stuck in the loop, handle it carefully. 
        3. Debug and correct your implementation until it returns a non empty valid list of publication 
            from semanticscholar for the given query.
            - Make sure that the returned publications matches the topic.
            - You many need to gradually expand the query if results are too small.
            - You may need to make alternative implementation if many attempts dont solve same bug.
        4. Remember, if there is a time consuming steps i.e. finding papers succeeds, you can store intermediate output in temporary file,
            to save times in debugging later steps. Its because to fix a bug in later steps, re running previous steps if ineffective.
        5. The final review document should look very professional and scientific. Links should be hyperlinked.
        6. After retrieving, format your findings as a structured report with clear sections and paper summaries.
        7. If number of papers are too many, you can summarise the abstracts. But make sure all papers are addressed.
        8. After retrieving, read all the papers summary and prepare the summary of all the publication works 
            within 10 sentences and append at the document. You can cite the papers in the summary text as IEEE bibliography style. 
            IMPORTANT: Make sure that in the References section, each reference seperated by multiple newline the .md (markdown),
            so that in the converted PDF it looks like IEEE bibliography styled.
        9. In the final review document, there should be,
            - 5 to 10 sentences introduction and 
            - 10 to 15 sentences summary section where all the reference paper works are summarized. Summary should not be made up, but based on actual paper works.
        10. The final document should be prepared in markdown format and should be saved as .md and .pdf with suitable name for the topic in the given folder.
    """,
    llm_config=gemini_llm_config,
    code_execution_config=code_execution_config,
)

def run_review_workflow(topic: str) -> str:
    groupchat = autogen.GroupChat(
        agents=[user_proxy, researcher],
        messages=[],
        max_round=20,
        speaker_selection_method="round_robin",
        allow_repeat_speaker=True
    )
    
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=gemini_llm_config
    )
    
    user_proxy.initiate_chat(
        manager,
        message=f"""
        1. You need to generate a comprehensive technical review document on "{topic}". 
        2. Ask Researcher to search the topic on arxiv and prepare the technical review document.
        2. After you get the final technical review document, implement create_pdf function which can convert the review document to pdf. 
        3. You may need to install dependent Python module i.e. markdown2, weasyprint and so on. to implement and debug your function until it returns valid PDF.
            - Create a new folder with suitable name for storing generated final documents for this topic.
        4. The final output should be a PDF file saved in the created folder.
        5. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
        """,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="NEVER",
    )
    
    return f"{topic.replace(' ', '_')}_review.pdf"

if __name__ == "__main__":
    topic = "Detecting HR using BCG and IMU"
    pdf_path = run_review_workflow(topic)
    print(f"Workflow completed. Review document (if generated) should be available at: {pdf_path}")

UserProxyAgent (to chat_manager):


        1. You need to generate a comprehensive technical review document on "Detecting HR using BCG and IMU". 
        2. Ask Researcher to search the topic on arxiv and prepare the technical review document.
        2. After you get the final technical review document, implement create_pdf function which can convert the review document to pdf. 
        3. You may need to install dependent Python module i.e. markdown2, weasyprint and so on. to implement and debug your function until it returns valid PDF.
            - Create a new folder with suitable name for storing generated final documents for this topic.
        4. The final output should be a PDF file saved in the created folder.
        5. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
        

--------------------------------------------------------------------------------

Next speaker: Researcher

Researcher (to chat_manager):

```python
fro